In [4]:
!pip install neptune

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple


In [125]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.datasets as datasets
import numpy as np
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from tqdm import tqdm

import neptune
from neptune.new.types import File
from utils import save_model

run = neptune.init_run(
    project="sben834520/gan-own-images",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhYzRmNTU1Ny1mMTEwLTQ4OWMtYTQxYy0xN2QwZDUwMGY3YjkifQ==",
)  # your credentials


batch_size = 1024
epochs = 100
sample_size = 64
latent_dim = 128
k = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
                               transforms.Resize(28),
                               transforms.Grayscale(1),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,), (0.5,)),
           ])

# train_data = datasets.MNIST(
#    root='/content/datasets',
#    train=True,
#    download=True,
#    transform=transform
# )

DATASET_PATH = "autodl-tmp/Duplicated_training_set"
train_data = datasets.ImageFolder(root=DATASET_PATH, transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

params = {"learning_rate": 0.0002,
         "optimizer": "Adam",
         "optimizer_betas": (0.5, 0.999),
         "latent_dim": latent_dim}

run["parameters"] = params
class Generator(nn.Module):
    def __init__(self, latent_space):
        super(Generator, self).__init__()
        self.latent_space = latent_space
        self.fcn = nn.Sequential(
           nn.Linear(in_features=self.latent_space, out_features=128*7*7),
           nn.LeakyReLU(0.2),
       )

        self.deconv = nn.Sequential(
           nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
           nn.LeakyReLU(0.2),

           nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
           nn.LeakyReLU(0.2),

           nn.Conv2d(in_channels=128, out_channels=1, kernel_size=(3, 3), padding=(1, 1)),
           nn.Tanh()
       )

    def forward(self, x):
        x = self.fcn(x)
        x = x.view(-1, 128, 7, 7)
        x = self.deconv(x)
        return x

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv = nn.Sequential(
           nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
           nn.LeakyReLU(0.2),

           nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
           nn.LeakyReLU(0.2)
       )
        self.classifier = nn.Sequential(
           nn.Linear(in_features= 3136, out_features=1),#3136 #16384
           nn.Sigmoid()
       )

    def forward(self, x):
       #print(f"input_disc : {x.shape}")
        x = self.conv(x)
       #print(f"conv_disc : {x.shape}")
        x = x.view(x.size(0), -1)
       #print(f"view_disc : {x.shape}")
        x = self.classifier(x)
       #print(f"output_disc : {x.shape}")
        return x
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0,0.02) 
#load model
def load_model(model, target_dir):
    model.load_state_dict(torch.load(f=target_dir))
    return model.to(device)

https://app.neptune.ai/sben834520/gan-own-images/e/GAN1-25


In [127]:
model_saved = "autodl-tmp/models/nep"


generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

initialize_weights(generator)
initialize_weights(discriminator)

load_model(generator,f"{model_saved}/generator/generator.pth")
load_model(discriminator,f"{model_saved}/discriminator/discriminator.pth")

optim_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optim_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

criterion = nn.BCELoss()
def label_real(size):
    labels = torch.ones(size, 1)
    return labels.to(device)
def label_fake(size):
    labels = torch.zeros(size, 1)
    return labels.to(device)
def create_noise(sample_size, latent_dim):
    return torch.randn(sample_size, latent_dim).to(device)

def train_generator(optimizer, data_fake):
    b_size = data_fake.size(0)
    real_label = label_real(b_size)
    optimizer.zero_grad()
    output = discriminator(data_fake)
    loss = criterion(output, real_label)
    loss.backward()
    optimizer.step()
    return loss

def train_discriminator(optimizer, data_real, data_fake):
    b_size = data_real.size(0)
   #print(f"b_size: {data_real.shape}")
    real_label = label_real(b_size)
    fake_label = label_fake(b_size)
    optimizer.zero_grad()
    output_real = discriminator(data_real)
    loss_real = criterion(output_real, real_label)
    output_fake = discriminator(data_fake)
    loss_fake = criterion(output_fake, fake_label)
    loss_real.backward()
    loss_fake.backward()
    optimizer.step()
    return loss_real, loss_fake
noise = create_noise(sample_size, latent_dim)

generator.train()
discriminator.train()


Discriminator(
  (conv): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.2)
  )
  (classifier): Sequential(
    (0): Linear(in_features=3136, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [ ]:
STEP_SIZE = 40
for epoch in range(epochs):
    loss_g = 0.0
    loss_d_real = 0.0
    loss_d_fake = 0.0    
    with tqdm(total=int(len(train_loader) / train_loader.batch_size), desc=f"Epoch {epoch}/{epochs}") as bar:
        for bi, data in enumerate(train_loader): #batchIdx, (real, _)
            #print(f"data: {data}")
            image, _ = data
            image = image.to(device)
            b_size = len(image)
            for step in range(k):
                data_fake = generator(create_noise(b_size, latent_dim)).detach()
                data_real = image
                loss_d_fake_real = train_discriminator(optim_d, data_real, data_fake)
                loss_d_real += loss_d_fake_real[0]
                loss_d_fake += loss_d_fake_real[1]
            data_fake = generator(create_noise(b_size, latent_dim))
            loss_g += train_generator(optim_g, data_fake)
           
            #inference and observations
            generated_img = generator(noise).cpu().detach()
            generated_img = make_grid(generated_img)
            generated_img = np.moveaxis(generated_img.numpy(), 0, -1)
            run[f'generated_img/{epoch}'].upload(File.as_image(generated_img))
            epoch_loss_g = loss_g / bi
            epoch_loss_d_real = loss_d_real/bi
            epoch_loss_d_fake = loss_d_fake/bi
            epoch_loss = (epoch_loss_d_real + epoch_loss_d_fake) / 2
            run["train/loss_generator"].log(epoch_loss_g)
            run["train/loss_discriminator_real"].log(epoch_loss_d_real)
            run["train/loss_discriminator_fake"].log(epoch_loss_d_fake)

            if bi % STEP_SIZE == 0 and bi != 0:                            
                try:
                    save_model(model=discriminator,target_dir=f"{model_saved}/nep/discriminator/",model_name=f"discriminator.pth")
                    save_model(model=generator,target_dir=f"{model_saved}/nep/generator/",model_name=f"generator.pth")
                except:
                    pass
            #print(f"Epoch {epoch}/{epochs} Loss D: {epoch_loss:.4f}, Loss G: {epoch_loss_g:.4f}")
            if bi % 10 == 0 and bi != 0:            
                bar.update(1)
                bar.desc = f"Epoch {epoch}/{epochs} Loss D: {epoch_loss:.4f}, Loss G: {epoch_loss_g:.4f}"

            #bar.update(1)
            #bar.desc = f"Epoch {epoch}/{epochs} Loss D: {15.66:.4f}, Loss G: {17.55:.4f}"

Epoch 0/100:   0%|          | 0/3 [00:00<?, ?it/s]

The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly valu

Error occurred during asynchronous operation processing: Cannot log infinite or NaN value to attribute train/loss_generator
Error occurred during asynchronous operation processing: Cannot log infinite or NaN value to attribute train/loss_discriminator_real
Error occurred during asynchronous operation processing: Cannot log infinite or NaN value to attribute train/loss_discriminator_fake


The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.


Epoch 0/100:  33%|███▎      | 1/3 [00:05<00:10,  5.45s/it]

...

The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly valu

Epoch 2/100 Loss D: 0.2866, Loss G: 1.9350: 298it [23:13,  4.62s/it]

The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly valu

Epoch 2/100 Loss D: 0.2866, Loss G: 1.9351: 299it [23:18,  4.62s/it]

The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly values in the array need to be in the [0, 1] range.
The smallest value in the array is -1.0. To be interpreted as colors correctly valu

In [ ]:
try:
    from google.colab import files
    files.download("/content/checkpoints.zip")
except:
    print("Not running in Google Colab, can't use google.colab.files.download()")